# Value-at-Risk

## Lecture 1 - Implementation with Inverse Cumulative Density Function  

In [1]:
import numpy as np
import yfinance as yf
from scipy.stats import norm
import pandas as pd
from datetime import datetime

In [2]:
def download_data(stock: str, start_date: datetime, end_date: datetime) -> pd.DataFrame:
    data = {}
    ticker = yf.download(stock, start_date, end_date)
    data[stock] = ticker["Adj Close"]
    return pd.DataFrame(data)


def calculate_var(position: float, mu: float, sigma: float, conf: float = 0.95) -> float:
    # this is the value-at-risk for tomorrow 
    var = position * (mu - sigma * norm.ppf(1 - conf))
    return var


def calculate_var_n(position: float, mu: float, sigma: float, n_days: int, conf: float = 0.95) -> float:
    # this is the value-at-risk for any future period
    var = position * (mu*n_days - sigma*np.sqrt(n_days) * norm.ppf(1 - conf))
    return var

In [3]:
# Citigroup Inc. Stock
stock = "C"
start_date = datetime(2014, 1, 1)
end_date = datetime(2018, 1, 1)

# download data
data = download_data(stock, start_date, end_date)

# compute log daily returns  
data["returns"] = np.log(data["C"] / data["C"].shift(1))
data = data[1:]
data

[*********************100%***********************]  1 of 1 completed


,C,returns
Date,,
2014-01-03,43.411251,0.021388
2014-01-06,43.744579,0.007649
2014-01-07,44.045357,0.006852
2014-01-08,44.557507,0.011561
2014-01-09,44.874561,0.007090
...,...,...
2017-12-22,63.072449,-0.004362
2017-12-26,62.479244,-0.009450
2017-12-27,62.571140,0.001470


In [4]:
# log returns parameter
mu = np.mean(data["returns"])
sigma = np.std(data["returns"])

# investment position $1Mio
S = 1e6

var = calculate_var(S, mu, sigma)
print(f"For an Investment of ${S:,} in CityGroup, tomorrow's VaR = ${var:,.2f}")

For an Investment of $1,000,000.0 in CityGroup, tomorrow's VaR = $25,396.73


In [5]:
n_days = 5
var = calculate_var_n(S, mu, sigma, n_days)
print(f"For an Investment of ${S:,} in CityGroup, {n_days} days VaR = ${var:,.2f}")

For an Investment of $1,000,000.0 in CityGroup, 5 days VaR = $57,834.34


## Lecture 2 - Implementation with Monte-Carlo Simulation

In [6]:
class ValueAtRiskMC:
    def __init__(self, P, mu, sigma, conf = 0.95, n_days = 1, iterations = 100_000) -> None:
        self.P = P
        self.mu = mu
        self.sigma = sigma
        self.conf = conf
        self.n_days = n_days
        self.iterations = iterations
    
    def simulation(self):
        rand = np.random.normal(0, 1, [1, self.iterations])

        # equation for the S(t) stock price
        stock_price = self.P * np.exp(
            self.n_days * (self.mu - 0.5 * self.sigma ** 2) \
                + self.sigma * np.sqrt(self.n_days) * rand
                )

        # we have to sort the stock prices to determine the percentile
        stock_price = np.sort(stock_price)

        # it depends on the confidence level: 95% -> 5 and 99% -> 1
        percentile = np.percentile(stock_price, (1 - self.conf) * 100)

        # VaR = S(t) - S'(t)
        return self.P - percentile

In [7]:
# investment position $1Mio
P = 1e6

# log returns parameter
mu = np.mean(data["returns"])
sigma = np.std(data["returns"])

model = ValueAtRiskMC(P, mu, sigma, conf=0.95, n_days=1)
var = model.simulation()

print(f"For an Investment of ${P:,} in CityGroup, tomorrow's VaR = ${var:,.2f}")

For an Investment of $1,000,000.0 in CityGroup, tomorrow's VaR = $24,488.52
